<a href="https://colab.research.google.com/github/nakamura196/tmp/blob/main/NDLOCR%E3%83%81%E3%83%A5%E3%83%BC%E3%83%88%E3%83%AA%E3%82%A2%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NDLOCRチュートリアル

画像のURLを引数として、OCRの認識結果を含むIIIFマニフェストファイルを生成します。

参考：青池さんのノートブック



## 使用方法

- まず、「1.初期セットアップ」を実行してください。初回のみ実行が必要です。

- その後は、「2. 設定」を変更して、OCRの適用対象のファイルを変更してください。

- 「3. 実行」を行うことで、認識結果が指定したパスに出力される他、推論実行後に表示されるURLをクリックすることで、推論結果（OCRの認識結果）をご確認いただけます。

## 1.初期セットアップ

（時間がかかります。初回のみ実行が必要です。）実行後は、「出力を消去」されることをお勧めします。

In [ ]:
# @title 初期セットアップ

# 1. NDLOCRのリポジトリをcloneする(--recursiveを忘れずに！)
!git clone -q --recursive https://github.com/ndl-lab/ndlocr_cli

# 2. 必要なパッケージをインストールする
PROJECT_DIR="/content/ndlocr_cli"
!pip install -r {PROJECT_DIR}/requirements.txt
!pip install -q torch==1.8.1+cu111 torchvision==0.9.1+cu111 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install -q mmcv-full==1.4.0 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.8.0/index.html
##numpyのバージョン問題でcolabでは動かなかったのでアップデートする(参考:https://stackoverflow.com/questions/66060487/valueerror-numpy-ndarray-size-changed-may-indicate-binary-incompatibility-exp)
!pip install -q --upgrade numpy

%cd {PROJECT_DIR}/src/ndl_layout/mmdetection
!python setup.py bdist_wheel
!pip install -q dist/*.whl
# %cd /content

# # 4. OCRに必要な学習済みモデルをダウンロードする

%cd {PROJECT_DIR}
!wget https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/mojilist_NDL.txt -P ./src/text_recognition/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/ndlenfixed64-mj0-synth1.pth -P ./src/text_recognition/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/ndl_layout_config.py -P ./src/ndl_layout/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/epoch_140_all_eql_bt.pth -P ./src/ndl_layout/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/separate_pages_ssd/weights.hdf5 -P ./src/separate_pages_ssd/ssd_tools

# 5. 環境変数を追加する
import os
os.environ["PYTHONPATH"]=os.environ["PYTHONPATH"]+":"+f"{PROJECT_DIR}/src/text_recognition/deep-text-recognition-benchmark"

# 6. google driveのマウント
from google.colab import drive
drive.mount('/content/drive/')

# 7. htmlの準備

# html
html = '''
<!DOCTYPE html>
<html lang="ja">
  <head>
    <link rel="icon" href="favicon.svg" type="image/svg+xml">
    <meta charset="utf-8" />
    <meta
      name="viewport"
      content="width=device-width, initial-scale=1, shrink-to-fit=no"
    />
    <meta name="theme-color" content="#000000" />
    <title>Mirador</title>
    <link
      rel="stylesheet"
      href="https://fonts.googleapis.com/css?family=Roboto:300,400,500"
    />
  </head>
  <body>
    <div
      id="mirador"
      style="position: absolute; top: 0; bottom: 0; left: 0; right: 0"
    ></div>
    <script>
      document.write(
        "<script type='text/javascript' src='https://unpkg.com/mirador@latest/dist/mirador.min.js'><\/script>"
      );
    </script>
    <script type="text/javascript">
      var windows = [];

      var manifest = "manifest.json"
      var obj = new Object();
      obj["manifestId"] = manifest;
      obj["thumbnailNavigationPosition"] = "far-right";
      windows.push(obj);

      var windowSettings = {
        highlightAllAnnotations: true,
        sideBarOpen: true,
        defaultSideBarPanel: 'annotations',
        allowClose: false,
        allowMaximize: false,
        allowFullscreen: true,
        hideWindowTitle: false,
      }

      var miradorInstance = Mirador.viewer({
        id: "mirador",
        language : "ja",
        window : windowSettings,
        windows,
      });
    </script>
  </body>
</html>
'''
f = open('/content/index.html', 'w')
f.write(html)
f.close()


PORT = 8000
PATH = ''

# 検証用に適当なサーバーを立ち上げておく
!nohup python3 -m http.server $PORT > server.log 2>&1 &

# インポート
from google.colab import output

## 2. 設定

OCRの適用対象のファイルを設定します。

- url: 画像のURL
- output_file_path: 出力するマニフェストファイルのパス
- process: 実行対象の処理を選択してください。（デフォルト：すべて）

In [ ]:
url = "https://www.dl.ndl.go.jp/api/iiif/3437686/R0000006/full/full/0/default.jpg" #@param {type:"string"}
output_file_path = "/content/drive/MyDrive/ndl_ocr/test.json" #@param {type:"string"}
process = "\u3059\u3079\u3066" #@param ["\u3059\u3079\u3066", "\u50BE\u304D\u88DC\u6B63,\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u30EC\u30A4\u30A2\u30A6\u30C8\u62BD\u51FA,\u6587\u5B57\u8A8D\u8B58(OCR)", "\u6587\u5B57\u8A8D\u8B58(OCR)"]

## 3. 実行

推論実行後に表示されるURLをクリックすることで、推論結果（OCRの認識結果）をご確認いただけます。

In [ ]:
#@title 推論の実行
from urllib import request
import glob
import tqdm
import bs4
import subprocess
from PIL import Image
import json

p = "0..3"
if process == "傾き補正,レイアウト抽出,文字認識(OCR)":
  p = "1..3"
elif process == "レイアウト抽出,文字認識(OCR)":
  p = "2..3"
elif process == "文字認識(OCR)":
  p = "3"

%cd {PROJECT_DIR}

def download_img(url, path):
  os.makedirs(os.path.dirname(path), exist_ok=True)
  request.urlretrieve(url, path)

def getImageSize(path):
  im = Image.open(path)
  # width, height = im.size
  return im.size

def getTmpOutputDir(path):
  return path.replace("/input/", "/output/")

def inference(path, output_dir):
  # 一時出力フォルダの準備
  filename = path.split("/")[-1]
  parent_dir = output_dir.replace(filename, "")
  if os.path.exists(parent_dir):
    !rm -rf $parent_dir
  os.makedirs(parent_dir, exist_ok=True)

  # 実行
  !python main.py infer -s f $path $output_dir -x -p $p

def createManifestFromXml(tmpOutputDir):
  # manifest生成
  xml_files = glob.glob(tmpOutputDir + "/**/*.xml", recursive=True)
  xml_files = sorted(xml_files)
  img = url
  items = []
  for i in range(len(xml_files)):
    file = xml_files[i]
    index = i + 1
    soup = bs4.BeautifulSoup(open(file), 'xml')
    element_names = ["LINE", "BLOCK"]
    pages = soup.find_all("PAGE")

    if len(pages) == 2:
      left_page_width = int(pages[0]["WIDTH"])

    annos = []

    canvas_id = "{}/{}".format(img, index)

    page_img = img

    item = {
        "annotations": [
          {
            "id": "{}/annos".format(canvas_id),
            "items": annos,
            "type": "AnnotationPage"
          }
        ],
        "height": height,
        "id": canvas_id,
        "items": [
          {
            "id": "{}/page".format(canvas_id),
            "items": [
              {
                "body": {
                  "format": "image/jpeg",
                  "height": height,
                  "type": "Image",
                  "width": width,
                  "id": page_img
                },
                "id": canvas_id + "/page/imageanno",
                "motivation": "painting",
                "target": canvas_id,
                "type": "Annotation"
              }
            ],
            "type": "AnnotationPage"
          }
        ],
        "label": { "none": ["[{}]".format(index)] },
        "type": "Canvas",
        "width": width,
        "thumbnail": [
          {
            "id": img, # page_img.replace(".jpg", "_r25.jpg"),
            "type": "Image",
            "format": "image/jpeg"
          }
        ]
      }
    items.append(item)

    for i in range(len(pages)):

      page = pages[i]

      page_height = page["HEIGHT"]
      page_width = page["WIDTH"]
      page_name = page["IMAGENAME"]

      for element_name in element_names:

        elements = page.find_all(element_name)

        for e in elements:
          try:
            conf = e["CONF"]
            text = e["STRING"]
            h = e["HEIGHT"]
            w = e["WIDTH"]
            x = e["X"]
            y = e["Y"]

            # 右ページの場合は要注意
            if i == 1:
              x = int(x) + left_page_width

            xywh = "{},{},{},{}".format(x, y, w, h)

            type_ = e["TYPE"]
            anno = {
              "id": canvas_id + "/annos/{}".format(len(annos)),
              "motivation": "commenting",
              "target": canvas_id + "#xywh={}".format(xywh),
              "type": "Annotation",
              "body": { "type": "TextualBody", "value": "【{}】{}".format(type_, text) }
            }
            annos.append(anno)
          except:
            # ルビ
            pass

  manifest = {
    "@context": "http://iiif.io/api/presentation/3/context.json",
    "type": "Manifest",
    "label": { "none": [id] },
    "items": items
  }

  return manifest

# 画像のダウンロード
path = "tmp/input/{}.jpg".format(id)
download_img(url, path)
# サイズの取得
width, height = getImageSize(path)
# 一時出力フォルダの作成
tmpOutputDir = getTmpOutputDir(path)
# 推論の実行
inference(path, tmpOutputDir)
# マニフェストファイルの作成
manifest = createManifestFromXml(tmpOutputDir)

# 出力フォルダに保存
output_path = output_file_path
os.makedirs(os.path.dirname(output_path), exist_ok=True)
json.dump(manifest, open(output_path, "w"))

# 確認用フォルダに保存
json.dump(manifest, open("/content/manifest.json", "w"), ensure_ascii=False,
            indent=4, sort_keys=True, separators=(',', ': '))

# 認識結果の確認
print("\n\n\n")
print("{} にマニフェストファイルを保存しました。".format(output_path))
print("また以下のリンクから、認識結果を確認できます。")
output.serve_kernel_port_as_window(PORT, path=PATH)